In [28]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import OnlineCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter, GrowingSpheres
from carla.models.negative_instances import predict_negative_instances
from carla.evaluation.benchmark import Benchmark
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from time import perf_counter

num = 10
data_name = "compas"

In [ ]:
dataset = OnlineCatalog(data_name)

In [ ]:
model = MLModelCatalog(dataset, "ann", backend="pytorch")
model.train(
    learning_rate = 0.001,
    epochs = 10,
    max_depth = 50,
    n_estimators = 50,
    batch_size = 20,
    force_train = True
)

In [ ]:
hyperparams = {
    "data_name": data_name,
    "train_vae": True,
    "width": 10,
    "depth": 3,
    "latent_dim": 12,
    "batch_size": 64,
    "epochs": 1,
    "lr": 0.001,
    "early_stop": 20,
}
# load a recourse model and pass black box model
cl = Clue(dataset, model, hyperparams)

wa = Wachter(model, {"loss_type": "BCE"})

In [ ]:
factuals = predict_negative_instances(model, dataset._df).sample(10)

wa_counterfactuals = wa.get_counterfactuals(factuals)
cl_counterfactuals = cl.get_counterfactuals(factuals)

In [ ]:
display(factuals.index)
display(cl_counterfactuals.index)
display(wa_counterfactuals.index)

In [30]:
gs = GrowingSpheres(model, dataset)

In [32]:
benchmark = Benchmark(model, gs, factuals)

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
benchmark.run_benchmark()

In [ ]:
factuals.index
model.feature_input_order

In [ ]:
df_cfs = pd.DataFrame(
    np.array(factuals[model.feature_input_order]), columns=model.feature_input_order, index=list(factuals.index)
)

In [ ]:
df_cfs.index = factuals.index

In [ ]:
df_cfs